In [ ]:
# 타임시퀀스가 늘어나며 역전파 시 하이퍼볼릭 탄젠트 함수의 미분값이 여러번 곱해져서
# 모델을 돌려도 어느 정도 이상부터는 결과가 한계에 부딪친다
# 즉 하이퍼볼린 탄젠트 함수를 미분하면 0에서 1사이의 값이 나오고
# 기울이 값이 역전파될 때 타임 시퀀스가 길어질수록 모델이 제대로 학습하지못함
# "기울기 소실" 이라는 현상
# 합성곱 신경망처럼 활성화 함수를 바꾸는 것도 하나의 개선 방법이지만
# 개선모델 => LSTM, GRU

In [ ]:
# LSTM : 기존이 순환신경망 모델에 장기기억을 담당하는 부분 추가

In [3]:
import torch

In [4]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_char = ', n_characters)

NameError: name 'string' is not defined

In [5]:
file = unidecode.unidecode(open('./data/input.txt').read())
file_len = len(file)
print('file_len = ', file_len)

NameError: name 'unidecode' is not defined

In [6]:
def random_chunk():
    start_index = random.radint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return tensor

print(char_tensor('ABCdef'))

def random_training_set():
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

NameError: name 'all_characters' is not defined

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_self = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.RNN(embedding_size, hidden_size, num_layers)    # GRU방식:RNN을 GRU로 바꾸기
        self.decoder = nn.Linear(hidden_size, output_size)
        
    def forward(sef, input, hidden):
        out = self.encoder(input.view(1, -1))
        out, hidden = self.rnn(out, hidden)
        out = self.decoder(out.view(batch_size, -1))
        return out, hidden
    
    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, batch_size, hidden_size)
        return hidden
    
model = RNN(
    input_size = n_characters,
    embedding_size = embedding_size,
    hidden_size = hidden_size,
    output_size = n_characters,
    num_layers = 2
)

In [ ]:
inp = char_tensor('A')
hidden = model.init_hidden()
out, hidden = model(inp, hidden)

for i in range(num_epochs):
    total = char_tensor(random_chunk())
    inp = total[:-1]
    label = total[1:]
    hidden = model.init_hidden()
    
    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()
    
    for j in range(chunk_len-1):
        x = inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y, hidden = model(x, hidden)
        loss += loss_func(y, y_)
        
    loss.backward()
    optimizer.step()
    

In [ ]:
# LSTM
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_self = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    def forward(sef, input, hidden, cell):
        out = self.encoder(input.view(batch_size, -1))
        out, (hidden, cell) = self.rnn(out, (hidden, cell))
        out = self.decoder(out.view(batch_size, -1))
        return out, hidden, cell
    
    def init_hidden(self):
        hidden = torch.zeros(num_layers, batch_size, hidden_size)
        cell= torch.zeros(num_layers, batch_size, hidden_size)
        return hidden
    
model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers)